<h1> Optimization electricity consumption </h1>

<h3> Imports </h3>

In [3]:
import math
import random
import copy
import numpy as np
import joblib

from preprocess_functions import *
from Individual import Individual

<h3> Helper functions </h3>

In [4]:
def calc_consumption_from_planing(plan,mat,day_consumption):
    for house in plan:
        for machine in house.keys():
            for time_slot in house[machine]:
                day_consumption[time_slot] += mat[machine]["power"]
    return day_consumption

<h3> Recuit functions </h3>

In [5]:
#Not Perfect
def initial_solution(empty_plannings,mat):

    for house in range(len(empty_plannings)):
        for machine in empty_plannings[house].keys():
            if type(mat[machine]["nb_time_slot"]) == list:
                mat[machine]["nb_time_slot"] = random.sample(mat[machine]["nb_time_slot"],1)[0]
            
            if machine in ['LL','SL','LV']:
                start_index = random.randint(0,47)
                for i in range(mat[machine]["nb_time_slot"]):
                    empty_plannings[house][machine].append((start_index+i)%48)
            else:
                if mat[machine]["is_sequencable"]:
                    for i in range(mat[machine]["nb_time_slot"]):
                        index = random.randint(0,15)
                        while index in empty_plannings[house][machine]:
                            index = random.randint(0,15)
                else:
                    start_index = random.randint(0,15-mat[machine]["nb_time_slot"])
                    for i in range(mat[machine]["nb_time_slot"]):
                        empty_plannings[house][machine].append(start_index+i)
                

    day_consumption = []
    for i in range(48):
        day_consumption.append(0)

    day_consumption = calc_consumption_from_planing(empty_plannings,mat,day_consumption)

    individual = Individual({"start":0,"end":15},empty_plannings,day_consumption)
    

    return individual

In [6]:
def get_neighbour_solution(solution: Individual):
    solution_neigh = copy.deepcopy(solution)
    solution_neigh.mutate()

    return solution_neigh

def cost_function(solution : Individual):
    return solution.getMax()

def metropolis(neighbour_solution, current_solution,temperature):
    return math.exp( -( abs( cost_function(current_solution) - cost_function(neighbour_solution) ) / temperature ) )

def update_temperature(temperature):
    return temperature * 0.99

In [7]:
def recuit_simule(temperature,current_solution):
    best_current_solution = copy.deepcopy(current_solution)
    print_count = 0
    count_iteration_without_improvment = 0

    while count_iteration_without_improvment < 1000:
        print_count += 1
        if print_count%10 == 0:
            print("\r",end="")
            print("**Recuit",print_count,count_iteration_without_improvment,end="                ")

        neighbour_solution = get_neighbour_solution(current_solution)

        if cost_function(neighbour_solution) < cost_function(current_solution) or random.random() < metropolis( neighbour_solution , current_solution , temperature):
            current_solution = neighbour_solution

        if cost_function(current_solution) < cost_function(best_current_solution):
            best_current_solution = copy.deepcopy(current_solution)
            count_iteration_without_improvment = 0
        else:
            count_iteration_without_improvment += 1

        temperature = update_temperature(temperature)
    
    return best_current_solution

<h3> Variables </h3>

In [8]:
matrice_type_puissance_par_demie_heure = {'LV':{"power":float(0.65),"nb_time_slot":2,"is_sequencable":False}, 
                                            'LL':{"power":float(1),"nb_time_slot":2,"is_sequencable":False}, 
                                            'SL':{"power":float(0.125),"nb_time_slot":8,"is_sequencable":False}, 
                                            'TV':{"power":float(0.05),"nb_time_slot":[1,2,3,4,5],"is_sequencable":False}, 
                                            'FG_1':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                                            'CE_1':{"power":float(0.18),"nb_time_slot":12,"is_sequencable":False}, 
                                            'CG':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                                            'FO':{"power":float(0.8),"nb_time_slot":[1,2],"is_sequencable":False}, 
                                            'PL':{"power":float(0.6),"nb_time_slot":[1,2],"is_sequencable":False}, 
                                            'FG_2':{"power":float(0.3),"nb_time_slot":4,"is_sequencable":False}, 
                                            'CE_2':{"power":float(0.25),"nb_time_slot":12,"is_sequencable":False}
                                            }

<h3> Main </h3>

In [9]:
data = load_csv("data.csv")
empty_planning = np.array(generate_empty_planning(data))
empty_planning = empty_planning[:100]

['Logement', 'LV', 'LL', 'SL', 'TV', 'FG_1', 'CE_1', 'CG', 'FO', 'PL', 'FG_2', 'CE_2']


In [10]:
print("Init Solution")
current_solution = initial_solution(empty_planning,matrice_type_puissance_par_demie_heure)
print("Score init",current_solution.getMax())

print("First iter")
best_found_solution = recuit_simule(1000, current_solution)

curr_best = cost_function(best_found_solution)

print("Enter While")
n=0
while  n < 3:
    if cost_function(best_found_solution) < curr_best:
        curr_best = cost_function(best_found_solution)
        print(best_found_solution.day_consumption)
        print(cost_function(best_found_solution))
    

    if n%1 == 0:
        print("iter n° :",n)
    
    n+=1
    best_found_solution = recuit_simule(1000,best_found_solution)

print(best_found_solution.day_consumption)
print(cost_function(best_found_solution))

Init Solution
Score init 93.92499999999994
First iter
**Recuit 27100 994                Enter While
iter n° : 0
**Recuit 1000 999                iter n° : 1
**Recuit 1000 999                iter n° : 2
**Recuit 1000 999                [69.55, 69.59, 69.45, 69.44, 69.58, 69.52, 69.62, 69.6, 69.59, 69.62, 69.59, 69.53, 69.61, 69.55, 69.38, 59.45, 9.26, 10.5, 10.44, 11.9, 16.98, 13.9, 10.39, 12.96, 11.91, 9.06, 8.31, 8.83, 10.24, 8.03, 5.27, 9.02, 9.53, 9.41, 8.95, 4.23, 6.6, 7.61, 9.66, 15.68, 14.65, 12.84, 14.09, 16.14, 18.5, 15.74, 7.3, 0.38]
69.62


In [11]:
joblib.dump(best_found_solution.plannings,"Plannings.joblib")
joblib.dump(best_found_solution.day_consumption,"Day_Consumption.joblib")
joblib.dump(best_found_solution,"Solution.joblib")

['Solution.joblib']